## Run a tokamak case with gvec and visualize the result

**Note:**

**This ipython notebook needs prior installation of the gvec python package within a virtual environment, which then should be chosen as kernel!**

In [ ]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

# OMP number of threads for gvec run needs to be before import of gvec
os.environ["OMP_NUM_THREADS"] = "2"
# needs `pip install` of gvec in virtual environment, and to be run in that environment!!!
import gvec  # using run & modifying the parameters & postprocessing

### Run gvec via python

In this example, we first define the parameters in a dictionary. See [GVEC user guide](https://gvec.readthedocs.io/latest/user/index.html) for the full list of parameters.

The parameters specify the input profiles (iota and pressure), boundary shape and discretization parameters.

- The three unknowns $X^1,X^2,\lambda$ are functions of the radial, poloidal and toroidal coordinates $\rho\in[0,1],\theta\in[0,2\pi],\zeta\in[0,2\pi]$, i.e. 
  $$X^1(\rho,\theta,\zeta),X^2(\rho,\theta,\zeta),\lambda(\rho,\theta,\zeta)$$
- The radial coordinate $\rho$ corresponds to the square-root of the normalized toroidal flux $\rho = \sqrt{\Phi / \Phi_\text{edge}}$.
- Note that the profiles are ***always given over the normalized toroidal flux***, i.e. $s=\rho^2$. 
- The default hmap (`which_hmap=1`) defines the map to cylinder coordinates, such that 
  $$(X^1,X^2,\zeta)\mapsto(R,Z,-\phi)$$
- The boundary description is defined by the real cosine/sine Fourier modes, for the unknowns $X^1(\theta,\zeta),X^2(\theta,\zeta)$ at $\rho=1$. 
- In this example, a simple elliptic tokamak is prescribed, meaning that the number of field periods is `nfp=1` and the number of toroidal modes is zero.
- GVEC is using B-splines for the radial discretization of the unknowns $X^1,X^2,\lambda$. 
    - The number of elements is the same for all quantities, which is defined by the parameter `sgrid_nElems`.
      Using at least two radial elements is recommended.  
    - The default grid spacing is uniform in $\rho$. 
    - The degree of the B-splines can defined separately, with `X1X2_deg`, `LA_deg`. 
      The resulting number of radial degrees of freedom for a B-Spline is `nElems + deg`. 

More details on the discretization can be found in the [theory section of the GVEC user guide](https://gvec.readthedocs.io/latest/user/theory.html).



In [ ]:
# define GVEC parameters in a dictionary:
params = {
    "ProjectName": "GVEC_elliptok",  #  used in output files
    # radial resolution parameters ----------------------------------------------------
    "sgrid_nElems": 5,  # number of radial B-spline elements
    "X1X2_deg": 5,  # degree of B-splines for X1 and X2
    "LA_deg": 5,  # degree of B-splines for LA
    # Fourier resolution parameters: --------------------------------------------------
    "X1_mn_max": [3, 0],  # maximum Fourier mode numbers for X1 [m:poloidal,n:toroidal]
    "X2_mn_max": [3, 0],  # maximum Fourier mode numbers for X2
    "LA_mn_max": [3, 0],  # maximum Fourier mode numbers for LA
    "nfp": 1,  # number of field periods
    # Fourier modes of boundary shape -------------------------------------------------
    "X1_b_cos": {
        (0, 0): 5.0,
        (1, 0): 0.9,
    },  # (m,n) cosine Fourier mode coefficient for X1 (=R)
    "X2_b_sin": {(1, 0): 1.1},  # (m,n) sine Fourier mode coefficient for X2 (=Z)
    # initial guess for magnetic axis (only (0,0) mode makes sense) -------------------
    "X1_a_cos": {(0, 0): 5.0},
    # profile definitions for iota and pressure ---------------------------------------
    "iota": {
        "type": "polynomial",  # c_0 + c_1*s + c_2*s^2 ..., s=rho^2
        "coefs": [0.625, 0.35],
    },
    "pres": {
        "type": "interpolation",
        "rho2": [0.0, 0.25, 0.5, 0.75, 1.0],  # rho^2 positions
        "vals": [1.0, 0.75, 0.5, 0.25, 0.0],  # pressure values at the rho^2 positions
        "scale": 1000.0,  # scaling factor
    },
    "PhiEdge": 1.0,  # total toriodal flux (scales magnetic field stength)
    # Minimizer parameters ------------------------------------------------------------
    "totalIter": 10000,  # maximum number of iterations
    "minimize_tol": 1e-8,  # abort tolerance on sqrt(|forces|^2) in the energy minimization
}

Now, we pass the parameter dictionary to `gvec.run`, with a run subdirectory `run_01`. There, by default, the final solution `{ProjectName}_State_final.dat` and the final set of parameters `parameter_{ProjectName}_final.ini`) is stored. 
We can extract and plot the history of the GVEC run. We see the change in total MHD energy and the norm of the forces, which are computed as $\sqrt{||f||^2}$ for each unknown $X^1,X^2,\lambda$.

In [ ]:
runpath = Path(f"run_{1:02d}")
run = gvec.run(params, runpath=runpath)  # overwrites runpath or creates a new one

fig = run.plot_diagnostics_minimization()

### Post-processing: evaluate the equilibrium

1. Load the final state file from the object `run` that was the output of `gvec.run`. 
   Alternatively, if run was done outside this notebook, we can find the state from the `runpath`.
2. Choose a discrete set of 1D points in radial direction `rho` (proportional to the square-root of the magnetic flux) and poloidal direction `theta` and toroidal direction `zeta`. 
   - Either by specifying an array, or just the number of points. 
   - As this example is a tokamak, only one point in toroidal direction is used. 
   
The data is stored in `ev`, which is a `xarray` dataset.

In [ ]:
state = run.state
# alternatively:
state = gvec.find_state(runpath)

rho = np.linspace(0, 1, 20)  # radial visualization points
theta = np.linspace(0, 2 * np.pi, 50)  # poloidal visualization points

ev = state.evaluate("X1", "X2", "LA", "iota", "p", rho=rho, theta=theta, zeta=[0.0])

### Visualize the result

Now, lets visualize the 1D profiles (input quantities) over the radial coordinate $\rho^2$, which equals the normalized to the toroidal flux $\Phi / \Phi_\text{edge}$.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

ax[0].plot(ev.rho**2, ev.iota)
ax[0].set(xlabel="$\\rho^2\sim$ tor. flux", title="iota profile")
ax[1].plot(ev.rho**2, ev.p)
ax[1].set(xlabel="$\\rho^2\sim$ tor. flux", title="pressure profile");

And visualize the 2D cross-section of the example.  Note that contours of $\rho$ and of the straight-field line angle $\theta^* =\theta +\lambda(\theta,\zeta)$  are shown.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

R = ev.X1[:, :, 0]
Z = ev.X2[:, :, 0]
R_axis = R[0, 0].item()
Z_axis = Z[0, 0].item()
rho_vis = R * 0 + ev.rho
theta_vis = R * 0 + ev.theta
thetastar_vis = ev.LA[:, :, 0] + ev.theta
p_vis = R * 0 + ev.p
rho_levels_vis = np.linspace(0, 1 - 1e-10, 11)
theta_levels_vis = np.linspace(0, 2 * np.pi, 16, endpoint=False)
c = ax.contourf(R, Z, p_vis, alpha=0.75)
fig.colorbar(c, ax=ax, label="pressure")
ax.contour(R, Z, rho_vis, rho_levels_vis, colors="black")
ax.contour(R, Z, thetastar_vis, theta_levels_vis, colors="red")
ax.set(
    xlabel="$R$",
    ylabel="$Z$",
    title=f"equilibrium solution, cross-section\n position of magnetic axis (R,Z)=({R_axis:.5f},{Z_axis:.5f})",
    aspect="equal",
    xlim=[0.8 * np.amin(R), 1.2 * np.amax(R)],
    ylim=[1.1 * np.amin(Z), 1.1 * np.amax(Z)],
)
ax.legend(
    handles=[
        plt.Line2D([0], [0], color="black", label="$\\rho=$const."),
        plt.Line2D([0], [0], color="red", label="$\\theta^*=$const"),
    ]
);

## pygvec command line interface

We can also work only on the command line with a parameter file.
Lets write the parameters from above to a `parameter.toml` file using

In [ ]:
gvec.util.write_parameters(params, "parameter.toml")

and then run `pygvec` on the command line (with the virtual environment activated) as 

   ```bash
   mkdir run_02
   cp parameter.toml run_02/.
   cd run_02
   export OMP_NUM_THREADS=2
   pygvec run parameter.toml
   ```
For postprocessing, we can go back to python then load the final state file written in the pygvec run, together with the parameterfile used:
```python
state = gvec.find_state("run_02")
```